In [ ]:
import networkx as nx
import numpy as np
from IPython.display import Image
import itertools as it
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Run this only in Colab environment
# from google.colab import drive
# drive.mount('/content/gdrive')
# path = '/content/gdrive/My Drive/<your_gdrivefolder>/data/'
###############################################################
# for local storage
path = '../data/'

# Chapter 2 Tutorial

Note that many exercises are followed by a block with some `assert` statements. These assertions may be preceded by some setup code. They are provided to give you feedback that you are on the right path -- receiving an `AssertionError` probably means you've done something wrong.

Contents:

1. Degree
2. Adjacency matrix
3. Paths
4. Directed paths and components
5. dataset: The Vienna-subway-network 

# 1. degree , average degree and degree distribution

Let's start with a very simple, undirected network.


In [ ]:
G = nx.Graph()

G.add_nodes_from(['a','b','c','d','e'])

G.add_edges_from([('a','b'),('b','c'),('a','c'),('a','d'),('c','e')])

nx.draw(G, with_labels=True)

the degree of a node (i.e. the number of connected neighbors) is often the most important node property

In [ ]:
# as a method:
print(dict(G.degree()))

In [ ]:
# as a function
print(dict(nx.degree(G)))

sorting the nodes according to their degree:

In [ ]:
# simple sorting of the dictionary only leads to a sorting of node names
print(sorted(dict(G.degree()).items()))


In [ ]:
# using the anonymous lambda function is a way to make the 2nd argument being the sorting key
# by default it sorts in ascending order 
# by setting reverse = True you can get the node with the highest degree first

print(sorted(dict(G.degree()).items(),key = lambda x: x[1], reverse = True))



degree distribution

In [ ]:
# counting the number of neighbors:
l_k = list(dict(G.degree()).values())
print('all occuring degrees: ', l_k)

# the set operation makes entries unique
s_k = set(l_k)
print('set of degrees', s_k)

# # counting the number of neighbors and store it into a dict
dict_k_frequency = {}
for k in s_k:
    dict_k_frequency[k] = l_k.count(k)

print('dictionary with degrees as keys and frequency as values: ', dict_k_frequency)

# note, for larger lists (N>1000) the .count method becomes inefficient
# use the Counter module instead




In [ ]:
# plot the degree distribution as a bar plot

plt.bar(dict_k_frequency.keys(), dict_k_frequency.values(),width=.4)
plt.xlabel('degree',fontsize = 16)
plt.ylabel('frequency',fontsize = 16)



# Exercise 1 (3pts)
Write a function that gets a Graph object as input and 

returns the average degree and the standard deviation -

test your function with the karate edgelist (see tutorial 0) 

plot the degree distribution of the karate network


In [ ]:
def avg_std_degree(G):
    ...
    


# 2. Adjacency matrix

there is a networkx function that comuputes the adjacency matrix directly 

In [ ]:
G = nx.Graph()
G.add_nodes_from(['a','b','c','d','e'])
G.add_edges_from([('a','b'),('b','c'),('a','c'),('a','d'),('c','e')])

nx.draw(G, with_labels=True)

adj_matrix = nx.adjacency_matrix(G)

print(adj_matrix)

In [ ]:
# The adjacency matrix has the scipy sparse type format to save memory: 
print(type(adj_matrix))

# plot it as a numpy matrix:

adj_matrix_np = adj_matrix.todense()
print(adj_matrix_np)

In [ ]:
# plot your adjacency matrix as a heatmap
plt.imshow(adj_matrix_np,cmap=plt.get_cmap('binary'))

In [ ]:
# it also works the other way around
# you can define a numpy matrix and convert it into a Graph object

N = 8
A = np.random.randint(2,size=(N,N))  # generate a matrix randomly filled with 0 and 1
A_symmetric = np.tril(A) + np.tril(A, -1).T # make it symmetric (to get an undirected network)
np.fill_diagonal(A_symmetric, 0) # write zeros on the diagonal to avoid selfloops

print(type(A_symmetric))
# print(A_symmetric)

In [ ]:

G = nx.from_numpy_array(A_symmetric)

nx.draw(G,
        with_labels=True,
        node_color='#d2323c',
        edge_color='#777777',
        node_size=300,
        font_color='white',
        font_size=12,
        )


In [ ]:
# same for directed network:

N = 8
A = np.random.randint(2,size=(N,N))  # generate a matrix randomly filled with 0 and 1
A[np.tril_indices(A.shape[0], -1)] = 0  # set the lower left triangle   

# A_symmetric = np.tril(A) + np.tril(A, -1).T # make it symmetric (to get an undirected network)
np.fill_diagonal(A, 0) # write zeros on the diagonal to avoid selfloops
print(A)

# make explicit that you want the network to be directed
G = nx.from_numpy_array(A,create_using=nx.DiGraph)

nx.draw(G,
        with_labels=True,
        node_color='#f8b100',
        edge_color='#333333',
        node_size=300,
        font_color='white',
        font_size=12,
        )



# 3. Paths



In [ ]:
G = nx.Graph()

G.add_nodes_from([1,2,3,4])

G.add_edges_from([(1,2),(2,3),(1,3),(1,4)])

nx.draw(G, with_labels=True)

A *path* in a network is a sequence of edges connecting two nodes. In this simple example, we can easily see that there is indeed at least one path that connects nodes 3 and 4. We can verify this with NetworkX:

In [ ]:
nx.has_path(G, 3, 4)

There can be more than one path between two nodes. Again considering nodes 3 and 4, there are two such "simple" paths:

In [ ]:
list(nx.all_simple_paths(G, 3, 4))

A simple path is one without any cycles. If we allowed cycles, there would be infinitely many paths because one could always just go around the cycle as many times as desired.

We are often most interested in *shortest* paths. In an unweighted network, the shortest path is the one with the fewest edges. We can see that of the two simple paths between nodes 3 and 4, one is shorter than the other. We can get this shortest path with a single NetworkX function:

In [ ]:
nx.shortest_path(G, 3, 4)

If you only care about the path length, there's a function for that too:

In [ ]:
nx.shortest_path_length(G, 3, 4)

Note that a path length is defined here by the number of *edges* in the path, not the number of nodes, which implies

    nx.shortest_path_length(G, u, v) == len(nx.shortest_path(G, u, v)) - 1
    
for nodes $u$ and $v$.

# Exercise 2 (3 pts)

o Write the adjacency matrix and the edgelist of the network from paper-exercise 3

o Compute the clustering coefficient, diameter and density

o Find the number of d=3 paths between 2 and 3

o Which node pair has the most d=3 paths? 


## Connected components

In a simple network, we can see that for *every* pair of nodes, we can find a path connecting them. This is the definition of a *connected* graph. We can check this property for a given graph:

In [ ]:
G = nx.Graph()

G.add_nodes_from([1,2,3,4])

G.add_edges_from([(1,2),(2,3),(1,3),(1,4)])

nx.draw(G, with_labels=True)

In [ ]:
nx.is_connected(G)

Not every graph is connected:

In [ ]:
G = nx.Graph()

nx.add_cycle(G,[1,2,3])
G.add_edge(4,5)

nx.draw(G, with_labels=True)

In [ ]:
nx.is_connected(G)

And NetworkX will raise an error if you ask for a path between nodes where none exists:

In [ ]:
nx.has_path(G, 3, 5)

In [ ]:
nx.shortest_path(G, 3, 5)

Visually, we can identify two connected components in our graph. Let's verify this:

In [ ]:
nx.number_connected_components(G)

The `nx.connected_components()` function takes a graph and returns a list of sets of node names, one such set for each connected component. Verify that the two sets in the following list correspond to the two connected components in the drawing of the graph above:

In [ ]:
list(nx.connected_components(G))

We often care about the largest connected component (lcc), which is sometimes referred to as the giant component. We can make use of Python's builtin `max` function in order to obtain the largest connected component. By default, Python's `max` function sorts things in lexicographic (i.e. alphabetical) order, which is not helpful here. We want the maximum connected component when sorted in order of their sizes, so we pass `len` as a key function:

In [ ]:
max(nx.connected_components(G), key=len)

generate the actual subgraph consisting of the largest connected component with the `G.subgraph()` function:

In [ ]:
core_nodes = max(nx.connected_components(G), key=len)
core = G.subgraph(core_nodes)

nx.draw(core, with_labels=True)

Those of you using tab-completion will also notice a `nx.connected_component_subgraphs()` function. This can also be used to get the core subgraph but the method shown is more efficient when you only care about the largest connected component.

# 4. Directed paths & components

Let's extend these ideas about paths and connected components to directed graphs.

In [ ]:
D = nx.DiGraph()
D.add_edges_from([
    (1,2),
    (2,3),
    (3,2), (3,4), (3,5),
    (4,2), (4,5), (4,6),
    (5,6),
    (6,4),
])
nx.draw(D, with_labels=True)

### Directed paths

We know that in a directed graph, an edge from an arbitrary node $u$ to an arbitrary node $v$ does not imply that an edge exists from $v$ to $u$. Since paths must follow edge direction in directed graphs, the same asymmetry applies for paths. Observe that this graph has a path from 1 to 4, but not in the reverse direction.

In [ ]:
nx.has_path(D, 1, 4)

In [ ]:
nx.has_path(D, 4, 1)

The other NetworkX functions dealing with paths take this asymmetry into account as well:

In [ ]:
nx.shortest_path(D, 2, 5)

In [ ]:
nx.shortest_path(D, 5, 2)

Since there is no edge from 5 to 3, the shortest path from 5 to 2 cannot simply backtrack the shortest path from 2 to 5 -- it has to go a longer route through nodes 6 and 4.

### Directed components

Directed networks have two kinds of connectivity. *Strongly connected* means that there exists a directed path between every pair of nodes, i.e., that from any node we can get to any other node while following edge directionality. Think of cars on a network of one-way streets: they can't drive against the flow of traffic.

In [ ]:
nx.is_strongly_connected(D)

*Weakly connected* means that there exist a path between every pair of nodes, regardless of direction. Think about pedestrians on a network of one-way streets: they walk on the sidewalks so they don't care about the direction of traffic.

In [ ]:
nx.is_weakly_connected(D)

If a network is strongly connected, it is also weakly connected. The converse is not always true, as seen in this example.

The `is_connected` function for undirected graphs will raise an error when given a directed graph.

In [ ]:
# This will raise an error
nx.is_connected(D)

In the directed case, instead of `nx.connected_components` we now have `nx.weakly_connected_components` and `nx.strongly_connected_components`:

In [ ]:
list(nx.weakly_connected_components(D))

In [ ]:
list(nx.strongly_connected_components(D))

# 5. The Vienna-subway-network 

In [ ]:
Image(path + 'U-Bahnnetz_Wien_2019.png')

In [ ]:
# for more complex datasets it is sometimes necessary to parse them yourself
# in order to construct a network

G_subway = nx.Graph()
f = open(path + 'Vienna_subway.csv','r')
lines = f.readlines()
for line in lines[1:]:
    start_node = line.strip().split(';')[0]
    end_node = line.strip().split(';')[1]
    color = line.strip().split(';')[3]
    G_subway.add_edge(start_node,end_node,color=color)
f.close()


In [ ]:
# a simple layout can be done by the networkx function spring_layout
# it basically considers the nodes as charged particles that repell each other 
# and edges as springs that attract connected nodes

print('# nodes: ', G_subway.number_of_nodes())
print('# edges: ', G_subway.number_of_edges())

colors = [G_subway[u][v]['color'] for u,v in G_subway.edges]

posG = nx.spring_layout(G_subway,iterations=200)
plt.figure(figsize=(15,15))
nx.draw_networkx(G_subway,pos = posG, node_size=50, node_color='#1f78b4',with_labels=1)
nx.draw_networkx_edges(G_subway,pos = posG,edge_color = colors,width=5)

# EXERCISE 3 (4+1 pts)

o Find the hub (station with the most connections) computationally

o Let's assume the subway always needs 2.5 minutes between the stations. What is the average traveling time for randomly chosen departure- and destination points? How long would the longest travel take (without detours)?

o Compute and plot the shortest path length distribution for the subway network.

o Write your own subway app: Make a function that get the subway network, a start and end station and returns the shortest connection. Try it out with start='Schoenbrunn' and end='Donauinsel'. To avoid running into to many tourists you want to get around 'Stephansdom'. How does that change your result?

o Advanced (voluntarily) - Perturbations/Vulnerabilities: Suppose your app is connected to a news feed that informs your system about temporally occuring traffic perturbations or station closures. Add a optional argument to your function that get a station to avoid. (+1 pt)